<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Fine-tune OpenAI GPT-3.5 Turbo Model in Teradata Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The large Language Model (LLM) is a general-purpose model designed for a broad range of NLP tasks, including providing information on various topics, answering questions, offering suggestions, and even helping us in our work.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>However, to use LLM in a highly specialized business use case scenario like finance or healthcare, we need to train the model using a specific dataset to refine its capabilities and improve its performance. Fine-tuning can achieve this.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Fine-tuning</b> is training a pretrained model on a small and targeted dataset to achieve a specific task. By doing this, users can improve their performance on that task while preserving their general language knowledge.</p>

<center><img src="images/fine-tuning.png" alt="Fine_tuning_process"  width=600 height=480/></center>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo notebook, we train the GPT-3.5 Turbo model using OpenAI API on the "mental health" dataset. By doing so, the model becomes a specialist in answering mental health-related questions and can provide responses on how to tackle mental health and related issues.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>OpenAI allows users to create their own custom GPT-3.5 model tuned towards a particular dataset. As per the use case, we can teach GPT-3.5 the language and terminology of our niche domain, such as medicine or finance. The ChatGPT models are available via API, and in the example below, we used GPT-3.5 Turbo.</p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Business Value</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Enhanced model performance for specific tasks.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Saves cost and time in model development.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Able to address real-world business problem in an efficient way. </li>
</p> 

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Why Vantage? </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage provides a platform for storing data required for fine-tuning. This data is then cleaned, formatted, and validated to meet the standards necessary for fine-tuning GPT-3.5 Turbo model. Once it is ready, the data is stored as training and validation JSONL files.</p> 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The new data files saved in Vantage are then used to fine-tune the GPT-3.5 Turbo model with OpenAI API. Once the model is fine-tuned, it can be tested on Vantage.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Connection to Vantage and OpenAI</li>
    <li>Data Exploration</li>
    <li>Data errors and cost estimation</li>
    <li>Fine-tuning the model</li>
    <li>Testing the fine-tuned model</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Install the required libraries</b></p>

In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages

!pip install -r requirements.txt --quiet

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <i>*The above statements will install the required libraries to run this demo. To gain access to installed libraries after running this, restart the kernel.</i></p>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><i><b>Note:</b> The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b>0 0</b></i></p></div>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Importing required libraries
import json
import os
import tiktoken # for token counting
import openai

# teradata lib
from teradataml import *

# Suppress warnings
warnings.filterwarnings("ignore")
display.print_sqlmr_query = False
display.suppress_vantage_runtime_warnings = True

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>2. Connection to Vantage</b>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Fine_Tuning_OpenAI_Model_Python.ipynb;' UPDATE FOR SESSION;''')

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.2 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can run the demo using foreign tables to access the data without any storage on your environment</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_LLM_FineTuning_cloud');"

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step. If you want, you can see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data used in the demostration is a mental health chatbot dataset containng questions and answers around the mental health issues. You can find the complete information of data <a href="https://huggingface.co/datasets/heliosbrahma/mental_health_chatbot_dataset">here</a>.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our database, we have one table: the <b>Mental_Health</b> table. The Mental_Health table contains text conversation between a human and an assistant about mental health issues. Human asks a question and assistant helps human with an answer.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.1 Examine the Mental Health table</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's look at the data in the table.</p>

In [ ]:
tdf_mental_health = DataFrame(in_schema("DEMO_LLM_FineTuning", "Mental_Health"))
print("Data information: \n", tdf_mental_health.shape)

pd_mental_health = tdf_mental_health.to_pandas()
print(pd_mental_health)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The table has 1 column: <b>text</b> and 172 rows. Each row contains a pair of a question and an answer related to mental health.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>3.2 Converting to correct format</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data in the table needs to be converted into a format suitable for fine-tuning OpenAI GPT-3.5 Turbo model.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The format required is:</p>

```json
{"messages": [{"role": "system", "content": "text"}, {"role": "user", "content": "text"}, {"role": "assistant", "content": "text"}]}
{"messages": [{"role": "system", "content": "text"}, {"role": "user", "content": "text"}, {"role": "assistant", "content": "text"}]}
{"messages": [{"role": "system", "content": "text"}, {"role": "user", "content": "text"}, {"role": "assistant", "content": "text"}]}
```

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The messages is a dictionary with two keys:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b>roles</b>: <i>system</i>, <i>user</i>, or <i>assistant</i> tells us from where content came from</li></ul>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li> <b>content</b>: the text content of the message</li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'></p>

In [ ]:
def conversation_converter(conversation_data, system_message=None):
    
    #Splitting the conversation string into individual lines
    lines = conversation_data.split('\n<')
    
    messages = []
    
    # Including the system message if provided
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })
    
    for l in lines:
        parts = l.split(': ', 1)
        # print(parts)
        
        if parts[0] == "<HUMAN>":
            role = "user"
        else:
            role = "assistant"
        
        message = {
            "role":role,
            "content":parts[1]
        }
        messages.append(message)
    
    output = {
        "messages":messages
    }
    return output

In [ ]:
system_message = """You are a helpful and understanding assistant who can help with mental health issues. You are friendly and polite"""

In [ ]:
data_mental_health = []

for i in pd_mental_health.index:
    data = pd_mental_health["text"][i]
    data_mental_health.append(data)

dataset = []

for data in data_mental_health:
    data = conversation_converter(data, system_message=system_message)
    dataset.append(data)

print("The length of data:", len(dataset))
print("An example from dataset:")
for message in dataset[0]["messages"]:
    print(message)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>*The output data looks to be in the correct format needed for fine-tuning.</p>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>4. Data errors and cost estimation</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before jumping to the fine-tuning, we need to make sure that the data format is correct and findout the cost estimation of fine-tuning process</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This part is taken from <a href="https://github.com/openai/openai-cookbook">openai_cookbook</a>.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.1 Format validation</b></p>

In [ ]:
#Format validation
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")


<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.2 Token Counting Utilities</b></p>

In [ ]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.3 Data Warnings and Token Counts</b></p>

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.4 Cost Estimation</b></p>

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>NOTE</b>: See the pricing page to check the cost as per the analysis result of previous step.</p>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>5. Fine-tuning the model</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After validating the data format, fine-tuning the model comes next.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.1 Get the OpenAI API key</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In order to fine-tune the model, you will need an OpenAI API key. If you do not have one, please refer to the instructions provided in this guide to obtain your OpenAI API key: </p>

[Openai_setup_api_key_guide](..//Openai_setup_api_key/Openai_setup_api_key.md)

In [ ]:
# enter your openai api key
api_key = input(prompt="\n Please Enter Openai api key: ")

os.environ["OPENAI_API_KEY"] = api_key

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.2 JSONL file for training</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>OpenAI accepts training files in JSONL format.</p>

In [ ]:
def write_jsonl(conversations, file_name):
    with open(file_name, 'w') as out:
        for conversation in conversations:
            json_out = json.dumps(conversation) + "\n"
            out.write(json_out)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first half of the dataset is used for training and the remaining will be used for validation. The validation data is optional which is used to make sure that the model does not overfit your training set. </p>

In [ ]:
training_file_name = 'training_data.jsonl'
validation_file_name = 'validation_data.jsonl'

# Training dataset
write_jsonl(dataset[:86], training_file_name)

# Validation dataset
write_jsonl(dataset[87:172], validation_file_name)

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.3 Upload files</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The files can be uploaded with the help of OpenAI's <b>Files</b> endpoint.</p>

In [ ]:
client = openai.OpenAI(api_key=api_key)

with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.4 Fine-tuning</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>With the generated files and suffix(optional) to identify model, we can create fine-tuning job. The <b>id</b> returned in response will help us retrieve updates of the job.</p>


    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>NOTE</b>: You might get <i>files not ready</i> error becasue the processing is done on OepnAI system. </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In case of an error, retry after few minutes.</p>

In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="mental-health",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

 <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>NOTE</b>: Run above cell only once or else the fine-tuning process will start again.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.5 Check job status</b></p>    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Initially the status will read as <i>running</i>, and will turn to <i>status: succeeded</i> after competion of job.</p>

In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Once it is completed, you can use the <b>result_files</b> to sample the results from the validation set (if you uploaded one), and use the ID from the <b>fine_tuned_model</b> parameter to invoke your trained model.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can track the progress of the fine-tune with the events endpoint.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Rerun the below cell few times until the fine-tune is ready.</p>

In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After completion of above cell, you will get a fine-tuned model ID from the job.</p>

In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>6. Testing the new model</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The last step is to use your fine-tuned model and test how it is working.</p>

In [ ]:
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = input(prompt="\n Please ask your question ")
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

<div id='section8'></div>
<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>7. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>7.1 Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
for table in ["Mental_Health"]:
    try:
        db_drop_table(table)

    except:
        pass

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'> <b>7.2 Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_LLM_FineTuning');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:28px;font-family:Arial;color:#00233c'>Links</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>OpenAI fine-tuning reference: <a href='https://platform.openai.com/docs/guides/fine-tuning'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>